In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
# import openpyxl
import statsmodels.api as sm

In [2]:
df = pd.read_excel('../data/OnlineRetailCampaign-2.xlsx', sheet_name='Data')
df.head()

,recency,historysegment,history,mens,womens,zipcode,newcustomer,channel,campaign,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [3]:
df.isnull().sum()

recency           0
historysegment    0
history           0
mens              0
womens            0
zipcode           0
newcustomer       0
channel           0
campaign          0
visit             0
conversion        0
spend             0
dtype: int64

In [4]:
def campaign_calculate(row):
    if row['campaign'] == 'No E-Mail':
        return 0
    elif row['campaign'] == 'Womens E-Mail':
        return 1
    elif row['campaign'] == 'Mens E-Mail':
        return 2
    else: 
        return None

df['campaign_dummy'] = df.apply(campaign_calculate, axis=1)

In [5]:
def channel_calculate(row):
    if row['channel'] == 'Phone':
        return 0
    elif row['channel'] == 'Web':
        return 1
    elif row['channel'] == 'Multichannel':
        return 2
    else: 
        return None

df['channel_dummy'] = df.apply(channel_calculate, axis=1)

In [6]:
df.head()

,recency,historysegment,history,mens,womens,zipcode,newcustomer,channel,campaign,visit,conversion,spend,campaign_dummy,channel_dummy
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0,1,0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0,1
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0,1,1
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,2,1
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0,1,1


In [13]:
# 'spend ~ history + womens + newcustomer + channel_dummy + campaign_dummy'
cons = sm.add_constant(df[['history', 'womens', 'newcustomer', 'channel_dummy', 'campaign_dummy']])
# X_exog = sm.add_constant(pd.get_dummies(cons,drop_first=True))
# X_exog['history'] = df['history']
# zi_model = sm.ZeroInflatedPoisson(df['spend'], exog=X_exog, exog_infil=df['spend'], dist='poisson').fit(maxiter=100, method='bfgs')
X_exog_infl = sm.add_constant(df[['spend']])  # or whatever features make sense
zi_model = sm.ZeroInflatedPoisson(df['spend'], exog=cons, exog_infl=None, dist='poisson').fit(maxiter=100, method='lbfgs')

print(zi_model.summary())

C:\Users\cbloom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:130: ValueWarning: unknown kwargs ['dist']
  warnings.warn(msg, ValueWarning)
C:\Users\cbloom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\cbloom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:130: ValueWarning: unknown kwargs ['dist']
  warnings.warn(msg, ValueWarning)


                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:                   spend   No. Observations:                64000
Model:             ZeroInflatedPoisson   Df Residuals:                    63994
Method:                            MLE   Df Model:                            5
Date:                 Wed, 02 Apr 2025   Pseudo R-squ.:                  -9.172
Time:                         22:41:15   Log-Likelihood:            -2.9116e+05
converged:                        True   LL-Null:                       -28624.
Covariance Type:             nonrobust   LLR p-value:                     1.000
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
inflate_const      0.1000        nan        nan        nan         nan         nan
const              0.0347        nan        nan        nan         nan         nan
history            0.0005   